> 토픽 모델링

- 문서 집합에 숨어 있는 주제를 찾아내는 것
- 많은 양의 문서가 있을 때, 모든 문서를 다 읽고 핵심 주제를 찾는 것은 매우 많은 시간이 소모됨
- 머신러닝의 토픽 모델링을 통해 숨어 있는 중요 주제를 효과적으로 찾아낼 수 있다.


1. **LSA (Latent Semantic Analysis)**:
   - LSA는 문서 간 및 단어 간의 의미적 유사성을 분석하여 토픽을 추출하는 기술입니다.
   - 주로 특이값 분해(Singular Value Decomposition, SVD)를 사용하여 단어-문서 행렬을 낮은 랭크의 근사 행렬로 분해합니다.
   - 이러한 근사 행렬을 사용하여 문서 간의 유사성 및 단어 간의 의미적 관계를 파악하고 토픽을 추출합니다.
   - LSA는 문서 간의 유사성과 단어 간의 의미적 유사성을 고려하여 토픽을 추출하므로 대체로 좀 더 전통적인 방식으로 토픽을 모델링합니다.

2. **LDA (Latent Dirichlet Allocation)**:
   - LDA는 문서의 토픽 분포와 단어의 토픽 분포를 확률적으로 모델링하여 토픽을 추출하는 기술입니다.
   - **각 문서가 여러 토픽의 혼합으로 구성되어 있다고 가정하고, 각 단어가 해당 문서의 특정 토픽에서 생성되었다고 가정합니다.**
   - 이러한 가정에 따라 문서의 토픽 분포와 단어의 토픽 분포를 추정하고, 이를 토픽으로 활용합니다.
   - LDA는 문서의 토픽 분포와 단어의 토픽 분포를 확률적으로 모델링하여 토픽을 추출하기 때문에 더 유연하고 확률적인 방식으로 토픽을 모델링합니다.

간단히 말해서, LSA는 단어-문서 행렬을 분해하여 의미적 유사성을 고려하여 토픽을 추출하는 반면, LDA는 확률적인 방식으로 문서의 토픽 분포와 단어의 토픽 분포를 모델링하여 토픽을 추출합니다.

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 모터사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 의학, 우주 주제를 추출.
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med']
news_df = fetch_20newsgroups(subset = 'all', remove = ('headers', 'footers', 'quotes'), categories = cats, random_state = 0)

count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english', ngram_range=(1,2))
feat_vect = count_vect.fit_transform(news_df.data)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (7862, 1000)


In [4]:
lda = LatentDirichletAllocation(n_components = 8, random_state = 0)
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=8, random_state=0)

In [5]:
# 각 주제마다 단어의 빈도 수
print(lda.components_.shape)
lda.components_

(8, 1000)


array([[3.60992018e+01, 1.35626798e+02, 2.15751867e+01, ...,
        3.02911688e+01, 8.66830093e+01, 6.79285199e+01],
       [1.25199920e-01, 1.44401815e+01, 1.25045596e-01, ...,
        1.81506995e+02, 1.25097844e-01, 9.39593286e+01],
       [3.34762663e+02, 1.25176265e-01, 1.46743299e+02, ...,
        1.25105772e-01, 3.63689741e+01, 1.25025218e-01],
       ...,
       [3.60204965e+01, 2.08640688e+01, 4.29606813e+00, ...,
        1.45056650e+01, 8.33854413e+00, 1.55690009e+01],
       [1.25128711e-01, 1.25247756e-01, 1.25005143e-01, ...,
        9.17278769e+01, 1.25177668e-01, 3.74575887e+01],
       [5.49258690e+01, 4.47009532e+00, 9.88524814e+00, ...,
        4.87048440e+01, 1.25034678e-01, 1.25074632e-01]])

- lda_model.components_ 값만으로는 각 토픽 별 word 연관도를 보기 어렵다.
- display_topics() 함수를 만들어서 각 토픽 별로 연관도가 높은 순으로 word를 나열

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

def display_topics(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환.
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]

        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_feature_names_out( )를 통해 추출
# argsort : 배열을 정렬한 후, 각 요소의 원래 인덱스를 반환하는 NumPy 배열 메서드 (기본 : 오름차순)
feature_names = count_vect.get_feature_names_out()

# Topic별 가장 연관도가 높은 word를 15개만 추출
display_topics(lda, feature_names, 15)

Topic # 0
year 10 game medical health team 12 20 disease cancer 1993 games years patients good
Topic # 1
don just like know people said think time ve didn right going say ll way
Topic # 2
image file jpeg program gif images output format files color entry 00 use bit 03
Topic # 3
like know don think use does just good time book read information people used post
Topic # 4
armenian israel armenians jews turkish people israeli jewish government war dos dos turkey arab armenia 000
Topic # 5
edu com available graphics ftp data pub motif mail widget software mit information version sun
Topic # 6
god people jesus church believe christ does christian say think christians bible faith sin life
Topic # 7
use dos thanks windows using window does display help like problem server need know run


> get_feature_names() 메서드는 scikit-learn의 CountVectorizer 클래스와 TfidfVectorizer 클래스의 메서드로, 각 특성(단어)의 이름(인덱스에 해당하는 단어)을 반환함